# 1) Aerodynamic and Aeroelastic Stability Derivatives Calculation using SHARPy

In this section, we will describe the process of trimming, linearising about the flight condition and evaluating the stability derivatives of the aircraft at said condition. This is a slightly more advanced analysis, as it needs specific settings for certain solvers in order for the solution to be correct.

In [1]:
import aircraft
import sharpy.utils.algebra as algebra
import numpy as np

It is assumed that the reader is familiar with generating the aicraft model files as in this chapter we will focus mostly on the settings required for the stability analysis. We commence by setting the flight conditions of interest.

In [2]:
# Simulation
case_name = 'hale_trim_derivatives'
cases_route = './cases/'
output_route = './output/'

# Flight conditions
u_inf = 10
rho = 1.225

# will be overwritten by Trim Routine
alpha = 3.4425 * np.pi / 180
elevator = -0.0991 * np.pi / 180
thrust = 4.2186

# Discretisation
m = 4
n_elem_multiplier = 1.5
wake_length = 10
horseshoe = True
tolerance = 1e-5


In [3]:
hale = aircraft.Hale(case_name, cases_route, output_route)

hale.clean()

hale.init_structure()
hale.init_aero(m)

hale.set_flight_controls(thrust=thrust, elevator=elevator)
hale.generate()

settings = dict()

The solvers required for this analysis are the same as those for a standard aircraft linearisation (recall we need a single timestep of `DynamicCoupled` to initialise the gyroscopic terms in the structural matrices and we need `Modal` to extract these matrices). We will be using the `StabilityDerivatives` post-processor once the linear system has been assembled.

In [4]:
# Derived parameters
dt = hale.aero.chord_main / m / u_inf

In [5]:
settings['SHARPy'] = {'case': hale.case_name,
                      'route': hale.case_route,
                      'flow': ['BeamLoader',
                               'AerogridLoader',
                                'StaticCoupled',
                                'DynamicCoupled',
                                'Modal',
                                'AeroForcesCalculator',
                                'LinearAssembler',
                                'StabilityDerivatives'
                              ],
                      'write_screen': 'on',
                      'write_log': 'on',
                      'log_folder': hale.output_route,
                      'log_file': hale.case_name + '.log'}

settings['BeamLoader'] = {'unsteady': 'on',
                          'orientation': algebra.euler2quat(np.array([0.,
                                                                      alpha,
                                                                      0.]))}

settings['AerogridLoader'] = {'unsteady': 'on',
                              'aligned_grid': 'on',
                              'mstar': wake_length * m,
                              'wake_shape_generator': 'StraightWake',
                              'wake_shape_generator_input': {
                                  'u_inf': u_inf,
                                  'u_inf_direction': [1., 0., 0.],
                                  'dt': dt,
                              },
                              }

## Nonlinear Aeroelastic Solvers

In [6]:
settings['NonLinearStatic'] = {'print_info': 'off',
                               'max_iterations': 150,
                               'num_load_steps': 1,
                               'delta_curved': 1e-1,
                               'min_delta': tolerance,
                               'gravity_on': 'on',
                               'gravity': 9.81,
                               'initial_position': [0., 0., 0.]}

settings['StaticUvlm'] = {'print_info': 'on',
                          'horseshoe': horseshoe,
                          'num_cores': 4,
                          'velocity_field_generator': 'SteadyVelocityField',
                          'velocity_field_input': {'u_inf': u_inf,
                                                   'u_inf_direction': [1., 0, 0]},
                          'rho': rho}

settings['StaticCoupled'] = {'print_info': 'off',
                             'structural_solver': 'NonLinearStatic',
                             'structural_solver_settings': settings['NonLinearStatic'],
                             'aero_solver': 'StaticUvlm',
                             'aero_solver_settings': settings['StaticUvlm'],
                             'max_iter': 100,
                             'n_load_steps': 1,
                             'tolerance': tolerance,
                             'relaxation_factor': 0.2}

settings['StaticTrim'] = {'solver': 'StaticCoupled',
                          'solver_settings': settings['StaticCoupled'],
                          'initial_alpha': alpha,
                          'initial_deflection': elevator,
                          'initial_thrust': thrust,
                          'fz_tolerance': 0.1,
                          'fx_tolerance': 0.1,
                          'm_tolerance': 0.1,
                          'save_info': 'on',
                          }

The `DynamicCoupled` solver is needed such that terms that depend on the rigid body velocity appear in the structural matrices. This is important for the aeroelastic stability derivatives. If only the aerodynamic (rigid) derivatives are desired, this step is not necessary since the aerodynamics are oblivious to the aircraft moving through the air or it being stationary in an oncoming flow. 

Thus, note that in this solver we enforce the aircraft to fly with a forward flight velocity and we set the external flow velocity to 0.

In [7]:
struct_solver_settings = {'print_info': 'off',
                          'initial_velocity_direction': [-1., 0., 0.],
                          'max_iterations': 950,
                          'delta_curved': 1e-6,
                          'min_delta': tolerance,
                          'newmark_damp': 5e-3,
                          'gravity_on': 'on',
                          'gravity': 9.81,
                          'num_steps': 1,
                          'dt': dt,
                          'initial_velocity': u_inf * 1}

step_uvlm_settings = {'print_info': 'on',
                      'num_cores': 4,
                      'convection_scheme': 2,
                      'vortex_radius': 1e-6,
                      'velocity_field_generator': 'SteadyVelocityField',
                      'velocity_field_input': {'u_inf': u_inf * 0,
                                               'u_inf_direction': [1., 0., 0.]},
                      'rho': rho,
                      'n_time_steps': 1,
                      'dt': dt,
                      'gamma_dot_filtering': 3}

settings['DynamicCoupled'] = {'print_info': 'on',
                              'structural_solver': 'NonLinearDynamicCoupledStep',
                              'structural_solver_settings': struct_solver_settings,
                              'aero_solver': 'StepUvlm',
                              'aero_solver_settings': step_uvlm_settings,
                              'fsi_substeps': 200,
                              'fsi_tolerance': tolerance,
                              'relaxation_factor': 0.3,
                              'minimum_steps': 1,
                              'relaxation_steps': 150,
                              'final_relaxation_factor': 0.5,
                              'n_time_steps': 1,
                              'dt': dt,
                              'include_unsteady_force_contribution': 'off'}

## Linear Solvers

We will extract the structural matrices at the resulting condition using the `Modal` solver. It is also necessary that we compute the aerodynamic forces thus we need the `AeroForcesCalculator` post-processor prior to linearising the aircraft.

In the `LinearAssembler` module we create the linearised system. This has been discussed in other tutorials but there are a few important notes to make in order to later on compute the stability derivatives:

LinearAssembler Notes:
  1. `['beam_settings']['discrete_time'] = 'off'`. The linear models in SHARPy have traditionally been assembled in discrete time since it is the natural way of linearising the UVLM system. However, since the stability analysis is a static analysis, assembling the beam in continuous time removes aliasing issues present in discrete systems near the Nyquist frequency and is capable of resolving features above this frequency. The Nyquist frequency is determined by the time step (usually determined by the UVLM panel discretisation) thus this avoids very refined aerodynamic grids and a more accurate solution overall.
  2. `['beam_settings']['use_euler'] = 'on'`. Another classic of the linear analysis, using the Euler angle parametrisation is recommended.
  3. `['beam_settings']['remove_rigid_states'] = 'on'`. This is __the most important__ setting in computing the stability derivatives. This removes the rigid states from the structural subsystem. Since the stability analysis is performed by perturbing the rigid body velocities, if these are also left as states, the system exhibits a zero behaviour at zero frequency in those transfer functions. Thus, to compute the aeroelastic derivatives, we just need to include the flexible degrees of freedom and we do so by removing these rigid states.
  4. `['aero_settings']['remove_inputs'] = ['u_gust']`. With these we remove all the external gust inputs which are not required for this particular analysis.
  5. `['aero_settings']['convert_to_ct'] = 'on'`. Given that the beam is assembled in continuous time we require the same to be done on the aero side. This is done using a bilinear transformation.

In [8]:
settings['AeroForcesCalculator'] = {'write_text_file': 'off'}


settings['Modal'] = {'rigid_modes_ppal_axes': 'on', # on or off. This changes the reference rigid body modes 
                                                     # (and derivatives) to be defined at the CG and aligned with
                                                     # the principal axes of inertia. Else, they are defined at the A
                                                     # frame reference.
                     'rigid_body_modes': 'on'}  # This will ensure the rigid body modes are captured, 
                                                 # else the analysis will assume a clamped structure

settings['LinearAssembler'] = {'linear_system': 'LinearAeroelastic',
                               'linear_system_settings': {
                                   'beam_settings': {'modal_projection': 'on',
                                                     'inout_coords': 'modes',  
                                                     'discrete_time': 'off',  # See Note LinearAssembler.1
                                                     'newmark_damp': 0.5e-3,
                                                     'discr_method': 'newmark',
                                                     'dt': dt,
                                                     'proj_modes': 'undamped',
                                                     'use_euler': 'on',  # See Note LinearAssembler.2
                                                     'num_modes': 20,
                                                     'print_info': 'on',
                                                     'gravity': 'on',
                                                     'remove_dofs': [],
                                                     'remove_rigid_states': 'on'  # See Note LinearAssembler.3
                                                    },
                                   'aero_settings': {'dt': dt,
                                                     'integr_order': 2,
                                                     'density': rho,
                                                     'remove_predictor': 'off',
                                                     'use_sparse': False,
                                                     'vortex_radius': 1e-7,
                                                     'remove_inputs': ['u_gust'],  # See note LinearAssembler.4
                                                     'convert_to_ct': 'on',  # See note LinearAssembler.5
                                                     },
                                   'track_body': 'off',
                                }
                               }

### Stability Derivatives post-processor

Finally, we just need to provide the normalisation scales to compute aerodynamic coefficients

In [9]:
settings['StabilityDerivatives'] = {'u_inf': u_inf,
                                    'c_ref': hale.aero.chord_main,
                                    'b_ref': hale.structure.span_main * 2,
                                    'S_ref': 2 * hale.structure.span_main * hale.aero.chord_main,
                                    }

In [10]:
hale.create_settings(settings)

In [11]:
hale.run()

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/ng213/2TB/KK_AirbusHALE/Delivery/01_StabilityDerivatives
SHARPy being run is in /home/ng213/2TB/KK_AirbusHALE/src/sharpy
The branch being run is airbus
The vers

/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/aero/utils/uvlmlib.py:264: RuntimeWarning: invalid value encountered in true_divide
  flightconditions.uinf_direction = np.ctypeslib.as_ctypes(ts_info.u_ext[0][:, 0, 0]/flightconditions.uinf)


|   1   | 0.0250 |  4   |   0.718453   |   1.007830   |  -5.539815   |-9.981812e+00 |-6.009100e-01 |
...Finished
Generating an instance of Modal
Variable print_info has no assigned value in the settings file.
    will default to the value: True
Variable use_undamped_modes has no assigned value in the settings file.
    will default to the value: True
Variable NumLambda has no assigned value in the settings file.
    will default to the value: 20
Variable write_modes_vtk has no assigned value in the settings file.
    will default to the value: True
Variable print_matrices has no assigned value in the settings file.
    will default to the value: False
Variable save_data has no assigned value in the settings file.
    will default to the value: True
Variable continuous_eigenvalues has no assigned value in the settings file.
    will default to the value: False
Variable dt has no assigned value in the settings file.
    will default to the value: 0.0
Variable delta_curved has no assigned

/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/solvers/modal.py:269: UserWarning: Projecting a system with damping on undamped modal shapes
  warnings.warn('Projecting a system with damping on undamped modal shapes')
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/structure/utils/modalutils.py:469: ComplexWarning: Casting complex values to real discards the imaginary part
  phi_rr[:, index_mode] = eigenvectors[-num_rigid_modes:, i]


|      0       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      1       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      2       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      3       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      4       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      5       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      6       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      7       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      8       |   0.000000   |   0.000000   |   0.000000   |   0.000000   |   1.000000   |     inf      |
|      9       |   0.000000   |   0.0

/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/linear/src/lingebm.py:1024: ComplexWarning: Casting complex values to real discards the imaginary part
  Ass[Nmodes:, :Nmodes] = -self.U.T.dot(self.Kstr.dot(self.U))
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/linear/src/lingebm.py:1026: ComplexWarning: Casting complex values to real discards the imaginary part
  Ass[Nmodes:, Nmodes:] = -self.Ccut[:Nmodes, :Nmodes]
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/linear/assembler/linearaeroelastic.py:264: ComplexWarning: Casting complex values to real discards the imaginary part
  in_mode_matrix[:2*beam.sys.num_dof, :2*beam.sys.num_modes] = sclalg.block_diag(phi, phi)


Aeroelastic system assembled:
	Aerodynamic states: 2184
	Structural states: 22
	Total states: 2206
	Inputs: 64
	Outputs: 60
Final system is:
State-space object
States: 2206
Inputs: 64
Outputs: 60

Input Variables:
	(InputVariable: q, size: 20, index: 0, starting at: 0, finishing at: 20)
	(InputVariable: q_dot, size: 20, index: 1, starting at: 20, finishing at: 40)
        (InputVariable: control_surface_deflection, size: 2, index: 2, starting
at: 40, finishing at: 42)
        (InputVariable: dot_control_surface_deflection, size: 2, index: 3,
starting at: 42, finishing at: 44)
	(InputVariable: Q, size: 20, index: 4, starting at: 44, finishing at: 64)
State Variables:
	(StateVariable: gamma, size: 168, index: 0, starting at: 0, finishing at: 168)
        (StateVariable: gamma_w, size: 1680, index: 1, starting at: 168,
finishing at: 1848)
        (StateVariable: dtgamma_dot, size: 168, index: 2, starting at: 1848,
finishing at: 2016)
        (StateVariable: gamma_m1, size: 168, index: 3, 

/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/utils/algebra.py:1383: ComplexWarning: Casting complex values to real discards the imaginary part
  res[0, 0] = v2*(sp*ss + cp*st*cp) + v3*(cp*ss - sp*st*cs)
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/utils/algebra.py:1384: ComplexWarning: Casting complex values to real discards the imaginary part
  res[0, 1] = v1*(-st*cs) + v2*(sp*ct*cs) + v3*(cp*ct*cs)
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/utils/algebra.py:1385: ComplexWarning: Casting complex values to real discards the imaginary part
  res[0, 2] = v1*(ct*ss) + v2*(-cp*cs - sp*st*ss) + v3*(sp*cs-cp*st*ss)
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/utils/algebra.py:1387: ComplexWarning: Casting complex values to real discards the imaginary part
  res[1, 0] = v2*(-sp*cs+cp*st*ss) + v3*(-cp*cs + sp*st*ss)
/home/ng213/2TB/KK_AirbusHALE/src/sharpy/sharpy/utils/algebra.py:1388: ComplexWarning: Casting complex values to real discards the imaginary part
  res[1, 1] = v1*(-s

Force/Angle via velocity



|==============|==============|==============|==============|
|     der      |     phi      |    alpha     |     beta     |
|==============|==============|==============|==============|
|      CD      |-1.802139e-11 | 7.601517e-02 |-5.610815e-08 |
|      CY      |-4.176822e-01 | 2.825099e-10 |-3.403444e-01 |
|      CL      |-3.144986e-10 | 5.791517e+00 |-1.333471e-06 |
|      Cl      |-3.638965e-04 |-1.334705e-10 | 2.411732e-01 |
|      Cm      | 4.408476e-09 |-3.324242e+00 | 1.320707e-05 |
|      Cn      |-6.048879e-03 | 4.966635e-10 |-4.306902e-02 |
|==============|==============|==============|==============|



Force derivatives to rigid body velocities - Body derivatives



|==============|==============|==============|==============|==============|==============|==============|
|     der      |      uA      |      vA      |      wA      |      pA      |      qA      |      rA      |
|==============|==============|==============|==============|=========

# Output

The output is written to a text file and also to an `.h5` file showing the stability derivatives (one for the aerodynamic (rigid) and one for the aeroelastic (flexible) derivatives).

### Aerodynamic Derivatives

In [12]:
with open(hale.output_route + hale.case_name + '/derivatives/aerodynamic_stability_derivatives.txt', 'r') as f:
    file = f.readlines()
    for line in file:
        print(line)

SHARPy Stability Derivatives Analysis

State:

	10.000000			 # Free stream velocity

	1.225000			 # Free stream density

	3.442726			 # Alpha [deg]

	0.000000			 # Beta [deg]



################################################################################

Centre of Gravity:

	x_A = 0.4491			 # [m]

	y_A = 0.0000			 # [m]

	z_A = 0.4303			 # [m]

Principal Axes Directions (expressed in the A frame):

	x_ppal in A = [0.9941, -0.0000, 0.1088]			

	y_ppal in A = [0.0000, 1.0000, -0.0000]			

	z_ppal in A = [-0.1088, 0.0000, 0.9941]			



################################################################################



Reference Dimensions:

	32.000000			 # Reference planform area

	1.000000			 # Reference chord

	32.000000			 # Reference span



################################################################################



Coefficients:

	2.615446e-03			 # CD

	-3.161342e-10			 # CY

	4.182796e-01			 # CL



#######################################################################

### Aeroelastic Derivatives

In [13]:
with open(hale.output_route + hale.case_name + '/derivatives/aeroelastic_stability_derivatives.txt', 'r') as f:
    file = f.readlines()
    for line in file:
        print(line)

SHARPy Stability Derivatives Analysis

State:

	10.000000			 # Free stream velocity

	1.225000			 # Free stream density

	3.442726			 # Alpha [deg]

	0.000000			 # Beta [deg]



################################################################################

Centre of Gravity:

	x_A = 0.4491			 # [m]

	y_A = 0.0000			 # [m]

	z_A = 0.4303			 # [m]

Principal Axes Directions (expressed in the A frame):

	x_ppal in A = [0.9941, -0.0000, 0.1088]			

	y_ppal in A = [0.0000, 1.0000, -0.0000]			

	z_ppal in A = [-0.1088, 0.0000, 0.9941]			



################################################################################



Reference Dimensions:

	32.000000			 # Reference planform area

	1.000000			 # Reference chord

	32.000000			 # Reference span



################################################################################



Coefficients:

	2.615446e-03			 # CD

	-3.161342e-10			 # CY

	4.182796e-01			 # CL



#######################################################################